# Data Cleaning

In [ ]:
import pandas as pd
import os

# Load the data from the correct directory
os.chdir(r"C:\Users\phili\Projects\immo-eliza-scraping-FireFlies\Data\Raw") # raw strings for syntax / input box for external user?
data = pd.read_csv("immoweb_raw_vf.csv")

# No blank spaces using strip() for strings data only
types = data.dtypes
for col in data.columns:
    if types[col] == 'string': # Apply str.strip() only to string columns
        data[col] = data[col].str.strip()

# Fill missing values with "None"
data = data.fillna("None")

# Filter out rows where the 'Price' column contains 'None' (2.744)
data = data[data['Price'] != 'None']
data = data[data['Habitable_Surface'] != 'None']

# Filter out the none existing postal code
data = data.loc[data['Postal_Code'] < 10000]

# Remove duplicate rows based on specified columns
data = data.drop_duplicates(subset=data.columns[1:], keep='first')

# Save the cleaned dataset in the correspondent directory
os.chdir(r"C:\Users\phili\Projects\immo-eliza-scraping-FireFlies\Data\Clean")
data.to_csv("immoweb_clean.csv", index=False)

# Data Analysis


## Conversion

In [ ]:
columns_to_convert = ['Price', 'Construction_Year', 'Bedroom_Count', 'Terrace_Surface','Garden_Surface', 'Land_Surface', 'Facade_Count']

# Convert specified columns to numeric, coercing errors to NaN
data[columns_to_convert] = data[columns_to_convert].apply(pd.to_numeric, errors='coerce')

placeholder_value = 0  
data['Facade_Count'] = data['Facade_Count'].fillna(placeholder_value)

# Convert columns to integer or float as needed
data['Construction_Year'] = data['Construction_Year'].astype(float)
data['Bedroom_Count'] = data['Bedroom_Count'].astype(int)
data['Postal_Code'] = data['Postal_Code'].astype(int)

# Information about the dataset
print(data.info())

## Add new columns

In [34]:
def assign_region(postal_code):
  """Assigns region based on the postal code digits."""
  if postal_code in range(1000, 1210):
    return 'Brussels'
  elif postal_code in range(1500, 1983) or postal_code // 1000 in [2, 3, 8, 9]:
    return 'Flanders'
  else:
    return 'Wallonia'
# Apply the region assignment function
data['Region'] = data['Postal_Code'].apply(assign_region)

# Creating subsets from price per m2
data['Price per m2'] = data['Price'] / data['Habitable_Surface']  

# Filter out rows that are too expensive
data = data[data['Price per m2'] < 40000]

#Create new categories according to price per m2
data['Price Category'] = pd.cut(
    data['Price per m2'],
    bins=[0, 2500, 5000, 10000, float("inf")],  # Price range boundaries, float("inf") represents infinity
    labels=["Value-driven investment", "Growth opportunities", "Established neighborhoods", "Premium properties"],
    right=True  # Include upper bound in each interval
)

data['Price Category']

0        Value-driven investment
2           Growth opportunities
3        Value-driven investment
4        Value-driven investment
5           Growth opportunities
                  ...           
59620       Growth opportunities
59621       Growth opportunities
59622       Growth opportunities
59623       Growth opportunities
59624       Growth opportunities
Name: Price Category, Length: 49680, dtype: category
Categories (4, object): ['Value-driven investment' < 'Growth opportunities' < 'Established neighborhoods' < 'Premium properties']

## Visualization

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as tck
import seaborn as sns

# Créer un tracé
sns.relplot(
  data=data,
  x="Price", y="Postal_Code", col="Type",
  hue="Subtype", size="Subtype"
)

# Formater les étiquettes de l'axe x (optionnel)
plt.gca().xaxis.set_major_formatter(tck.FuncFormatter(
  lambda x, pos: f"{x / 1000000:.1f}M"
))

# Afficher le tracé
plt.show()

# # Example: Boxplot to visualize the distribution of 'Price' for different 'Type' of properties
# plt.figure(figsize=(10, 6))
# sns.boxplot(x='Type', y='Price', data=data)
# plt.xticks(rotation=45)
# plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming your DataFrame has columns 'price_€_m2', 'investment_type', and 'region'
sns.histplot(
    data=data,
    x="Price per m2",
    y="Region",
    hue="Price Category",
    kde=True,  # Add density curves for smoother representation
    common_bins=True,  # Use consistent bins for comparison
)
plt.xticks(rotation=90)

# Add a title and axis labels
plt.title("Price Distribution by Investment Type and Region (Combined)")
plt.xlabel("Price (€/m²)")

plt.subplots_adjust(bottom=0.2)  # Adjust layout for better visibility

plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Get unique investment types
investment_types = data['Price Category'].unique()

# Create subplots and figure
fig, axes = plt.subplots(len(investment_types), 1, figsize=(10, 10))

# Iterate over investment types
for i, investment_type in enumerate(investment_types):
    df_filtered = data[data['Price Category'] == investment_type]  # Filter data

    sns.histplot(
        data=df_filtered,
        x="Price per m2",
        hue="Region",
        kde=True,
        common_bins=True,
        ax=axes[i]
    )

    # Subplot titles
    axes[i].set_title(investment_type)

# Shared title for all subplots
plt.suptitle("Price Distribution by Investment Type and Region")

# Adjust spacing and show
plt.tight_layout()
plt.show()


In [ ]:
# Select only numeric columns for the heatmap
numeric_data = data.select_dtypes(include='number')

# Calculate the correlation matrix
correlation_matrix = numeric_data.corr()

# Create a heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap of Numeric Features')
plt.show()

In [37]:
# Filter for apartments only
data_apartment = data[data['Type'] == "APARTMENT"]

# Group by Price Category and Region, then calculate mean and median
grouped_data = data_apartment.groupby(["Price Category", "Region"]).agg({
    "Price": ["mean", "median"],
    "Habitable_Surface": ["mean", "median"],
    "Bedroom_Count": ["mean", "median"],
})

print(grouped_data)

# Filter for apartments only
data_apartment = data[data['Type'] == "HOUSE"]

# Group by Price Category and Region, then calculate mean and median
grouped_data = data_apartment.groupby(["Price Category", "Region"]).agg({
    "Price": ["mean", "median"],
    "Habitable_Surface": ["mean", "median"],
    "Bedroom_Count": ["mean", "median"],
})

print(grouped_data)

C:\Users\phili\AppData\Local\Temp\ipykernel_1420\4169504409.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_data = data_apartment.groupby(["Price Category", "Region"]).agg({


                                           Price            Habitable_Surface  \
                                            mean     median              mean   
Price Category            Region                                                
Value-driven investment   Brussels  2.581289e+05   229000.0        132.268608   
                          Flanders  2.408016e+05   209000.0        126.097314   
                          Wallonia  1.870331e+05   170000.0        150.425341   
Growth opportunities      Brussels  3.597393e+05   299000.0        101.009492   
                          Flanders  3.410485e+05   307100.0         99.186936   
                          Wallonia  3.017206e+05   265000.0         92.984625   
Established neighborhoods Brussels  7.663215e+05   559000.0        119.780063   
                          Flanders  6.397940e+05   575000.0         94.305868   
                          Wallonia  4.081731e+05   299000.0         72.921569   
Premium properties        Br